In [195]:
import pandas as pd
import os

Lo que haremos aquí es recopilar los datos de las reviews de los estados que utilizaremos en el análisis. Los archivos en crudo se presentaron en formato Json, en archivos individuales para cada uno de los 51 estados de EEUU, por lo que procederemos a ordenar los mismos, en formato parquet, y por la dimension, en dataframes individuales, para luego crear una tabla única para los 5 estados pertinentes.

# New York

Se convierten los archivos Json a formato parquet

In [196]:
# Carpeta de entrada que contiene los archivos JSON
input_folder = r'C:\Users\debor\OneDrive\Documentos\Henry\PROYECTO FINAL local\Datasets\datasets_google\estados_json\NORESTE\review-New_York'
# Carpeta de salida para guardar los archivos Parquet
output_folder = r'C:\Users\debor\OneDrive\Documentos\Henry\PROYECTO FINAL local\Datasets\datasets_google\estados_parquet\NORESTE_parquet\Rev-New_York.parquet'

# Crear la carpeta de salida si no existe
os.makedirs(output_folder, exist_ok=True)

# Iterar sobre todos los archivos en la carpeta de entrada
for filename in os.listdir(input_folder):
    if filename.endswith('.json'):
        # Ruta completa del archivo JSON
        json_file_path = os.path.join(input_folder, filename)
        
        # Leer el archivo JSON
        try:
            df = pd.read_json(json_file_path, lines=True)
            
            # Crear nombre de archivo Parquet (sin la extensión .json)
            parquet_file_path = os.path.join(output_folder, filename.replace('.json', '.parquet'))
            
            # Guardar el DataFrame como archivo Parquet
            df.to_parquet(parquet_file_path, index=False, compression='snappy', engine='pyarrow')
            
            print(f"Convertido: {filename} a {parquet_file_path}")
        except Exception as e:
            print(f"Error al convertir {filename}: {e}")

print("Conversión completada.")

Convertido: 1.json a C:\Users\debor\OneDrive\Documentos\Henry\PROYECTO FINAL local\Datasets\datasets_google\estados_parquet\NORESTE_parquet\Rev-New_York.parquet\1.parquet
Convertido: 10.json a C:\Users\debor\OneDrive\Documentos\Henry\PROYECTO FINAL local\Datasets\datasets_google\estados_parquet\NORESTE_parquet\Rev-New_York.parquet\10.parquet
Convertido: 11.json a C:\Users\debor\OneDrive\Documentos\Henry\PROYECTO FINAL local\Datasets\datasets_google\estados_parquet\NORESTE_parquet\Rev-New_York.parquet\11.parquet
Convertido: 12.json a C:\Users\debor\OneDrive\Documentos\Henry\PROYECTO FINAL local\Datasets\datasets_google\estados_parquet\NORESTE_parquet\Rev-New_York.parquet\12.parquet
Convertido: 13.json a C:\Users\debor\OneDrive\Documentos\Henry\PROYECTO FINAL local\Datasets\datasets_google\estados_parquet\NORESTE_parquet\Rev-New_York.parquet\13.parquet
Convertido: 14.json a C:\Users\debor\OneDrive\Documentos\Henry\PROYECTO FINAL local\Datasets\datasets_google\estados_parquet\NORESTE_parq

In [197]:
# Leer el archivo Parquet
Rev_New_York_1 = pd.read_parquet(r'C:\Users\debor\OneDrive\Documentos\Henry\PROYECTO FINAL local\Datasets\datasets_google\estados_parquet\NORESTE_parquet\Rev-New_York.parquet')

print(Rev_New_York_1.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2700000 entries, 0 to 2699999
Data columns (total 8 columns):
 #   Column   Dtype  
---  ------   -----  
 0   user_id  float64
 1   name     object 
 2   time     int64  
 3   rating   int64  
 4   text     object 
 5   pics     object 
 6   resp     object 
 7   gmap_id  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 164.8+ MB
None


Se concatenan los archivos parquet para formar un dataframe

In [198]:
# Carpeta que contiene los archivos Parquet
input_folder_parquet = r'C:\Users\debor\OneDrive\Documentos\Henry\PROYECTO FINAL local\Datasets\datasets_google\estados_parquet\NORESTE_parquet\Rev-New_York.parquet'

# Lista para almacenar DataFrames
df_list_New_York = []

# Iterar sobre todos los archivos Parquet en la carpeta de entrada
for filename in os.listdir(input_folder_parquet):
    if filename.endswith('.parquet'):
        # Ruta completa del archivo Parquet
        parquet_file_path_ny = os.path.join(input_folder_parquet, filename)
        
        # Leer el archivo Parquet y añadir el DataFrame a la lista
        df = pd.read_parquet(parquet_file_path_ny)
        df_list_New_York.append(df)

# Concatenar todos los DataFrames en uno solo
df_new_york = pd.concat(df_list_New_York, ignore_index=True)

# Verificar el DataFrame combinado
print(df_new_york.shape)
print

(2700000, 8)


<function print(*args, sep=' ', end='\n', file=None, flush=False)>

Se elimina columna irrelevante para el análisis

In [199]:
df_new_york = df_new_york.drop(columns=['pics'])

Se convierten las columnas necesarias

In [200]:
# Se convierte la columna 'user_id' a tipo object
df_new_york['user_id'] = df_new_york['user_id'].astype('str')
print(df_new_york['user_id'].head(3))

0    1.1372210469230823e+20
1    1.0729344149210933e+20
2     1.003785858018194e+20
Name: user_id, dtype: object


In [201]:
# Se convierte la columna 'time' a datetime y luego a solo fecha manteniendo tipo datetime
df_new_york['time'] = pd.to_datetime(df_new_york['time'], unit='ms').dt.normalize()
print(df_new_york['time'].head(3))

0   2020-10-23
1   2021-05-04
2   2020-08-14
Name: time, dtype: datetime64[ns]


In [202]:
# Se normaliza la columna 'resp' para agregar su contenido como dos nuevas columnas a df_new_york
resp_df_ny = pd.json_normalize(df_new_york['resp'])
df_new_york[['resp_text', 'resp_time']] = resp_df_ny[['text', 'time']]

In [203]:
# Se convierte la columna 'resp_time' a datetime y luego a solo fecha manteniendo tipo datetime
df_new_york['resp_time'] = pd.to_datetime(df_new_york['resp_time'], unit='ms').dt.normalize()
print(df_new_york['resp_time'].head(3))

0          NaT
1   2021-05-06
2          NaT
Name: resp_time, dtype: datetime64[ns]


In [204]:
# Se elimina la columna 'resp' original
df_new_york = df_new_york.drop(columns=['resp'])

In [205]:
# Se renombra la columna gmap_id como 'id_negocio', para mayor comprensión en el análisis
df_new_york = df_new_york.rename(columns={'gmap_id': 'id_negocio'})

In [206]:
# Se reordena la columna 'id_negocio'
columnas = list(df_new_york.columns)
columnas.insert(0, columnas.pop(columnas.index('id_negocio')))
df_new_york = df_new_york[columnas]

In [207]:
# Se añade la columna 'review_id' autoincremental
df_new_york.insert(0, 'review_id', range(1, 1 + len(df_new_york)))

In [208]:
# Se añade la columna 'Estado'
df_new_york.insert(2, 'estado', 'New York')

In [209]:
# Se verifica como queda el dataframe
print(df_new_york.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2700000 entries, 0 to 2699999
Data columns (total 10 columns):
 #   Column      Dtype         
---  ------      -----         
 0   review_id   int64         
 1   id_negocio  object        
 2   estado      object        
 3   user_id     object        
 4   name        object        
 5   time        datetime64[ns]
 6   rating      int64         
 7   text        object        
 8   resp_text   object        
 9   resp_time   datetime64[ns]
dtypes: datetime64[ns](2), int64(2), object(6)
memory usage: 206.0+ MB
None


# California

Se pasan de formato Json a Parquet

In [210]:
# Carpeta de entrada que contiene los archivos JSON
input_folder = r'C:\Users\debor\OneDrive\Documentos\Henry\PROYECTO FINAL local\Datasets\datasets_google\estados_json\OESTE\review-California'
# Carpeta de salida para guardar los archivos Parquet
output_folder = r'C:\Users\debor\OneDrive\Documentos\Henry\PROYECTO FINAL local\Datasets\datasets_google\estados_parquet\OESTE.parquet\Rev-California.parquet'

# Crear la carpeta de salida si no existe
os.makedirs(output_folder, exist_ok=True)

# Iterar sobre todos los archivos en la carpeta de entrada
for filename in os.listdir(input_folder):
    if filename.endswith('.json'):
        # Ruta completa del archivo JSON
        json_file_path = os.path.join(input_folder, filename)
        
        # Leer el archivo JSON
        try:
            df = pd.read_json(json_file_path, lines=True)
            
            # Crear nombre de archivo Parquet (sin la extensión .json)
            parquet_file_path = os.path.join(output_folder, filename.replace('.json', '.parquet'))
            
            # Guardar el DataFrame como archivo Parquet
            df.to_parquet(parquet_file_path, index=False, compression='snappy', engine='pyarrow')
            
            print(f"Convertido: {filename} a {parquet_file_path}")
        except Exception as e:
            print(f"Error al convertir {filename}: {e}")

print("Conversión completada.")

Convertido: 1.json a C:\Users\debor\OneDrive\Documentos\Henry\PROYECTO FINAL local\Datasets\datasets_google\estados_parquet\OESTE.parquet\Rev-California.parquet\1.parquet
Convertido: 10.json a C:\Users\debor\OneDrive\Documentos\Henry\PROYECTO FINAL local\Datasets\datasets_google\estados_parquet\OESTE.parquet\Rev-California.parquet\10.parquet
Convertido: 11.json a C:\Users\debor\OneDrive\Documentos\Henry\PROYECTO FINAL local\Datasets\datasets_google\estados_parquet\OESTE.parquet\Rev-California.parquet\11.parquet
Convertido: 12.json a C:\Users\debor\OneDrive\Documentos\Henry\PROYECTO FINAL local\Datasets\datasets_google\estados_parquet\OESTE.parquet\Rev-California.parquet\12.parquet
Convertido: 13.json a C:\Users\debor\OneDrive\Documentos\Henry\PROYECTO FINAL local\Datasets\datasets_google\estados_parquet\OESTE.parquet\Rev-California.parquet\13.parquet
Convertido: 14.json a C:\Users\debor\OneDrive\Documentos\Henry\PROYECTO FINAL local\Datasets\datasets_google\estados_parquet\OESTE.parque

In [211]:
# Leer el archivo Parquet
Rev_California_1 = pd.read_parquet(r'C:\Users\debor\OneDrive\Documentos\Henry\PROYECTO FINAL local\Datasets\datasets_google\estados_parquet\OESTE.parquet\Rev-New_Mexico.parquet')

# Mostrar las primeras filas
print(Rev_California_1.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1800000 entries, 0 to 1799999
Data columns (total 8 columns):
 #   Column   Dtype  
---  ------   -----  
 0   user_id  float64
 1   name     object 
 2   time     int64  
 3   rating   int64  
 4   text     object 
 5   pics     object 
 6   resp     object 
 7   gmap_id  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 109.9+ MB
None


Se concatenan los archivos .parquet para formar un Dataframe

In [212]:
# Carpeta que contiene los archivos Parquet
input_folder_parquet = r'C:\Users\debor\OneDrive\Documentos\Henry\PROYECTO FINAL local\Datasets\datasets_google\estados_parquet\OESTE.parquet\Rev-California.parquet'

# Lista para almacenar DataFrames
df_list_California = []

# Iterar sobre todos los archivos Parquet en la carpeta de entrada
for filename in os.listdir(input_folder_parquet):
    if filename.endswith('.parquet'):
        # Ruta completa del archivo Parquet
        parquet_file_path_nmex = os.path.join(input_folder_parquet, filename)
        
        # Leer el archivo Parquet y añadir el DataFrame a la lista
        df = pd.read_parquet(parquet_file_path_nmex)
        df_list_California.append(df)

# Concatenar todos los DataFrames en uno solo
df_california = pd.concat(df_list_California, ignore_index=True)

# Verificar el DataFrame combinado
print(df_california.shape)
print

(2700000, 8)


<function print(*args, sep=' ', end='\n', file=None, flush=False)>

Se elimina columna irrelevante para el análisis

In [213]:
df_california = df_california.drop(columns=['pics'])

Se convierten las columnas necesarias

In [214]:
# Se convierte la columna 'user_id' a tipo object
df_california['user_id'] = df_california['user_id'].astype('str')
print(df_california['user_id'].head(3))


0    1.0899115226265579e+20
1    1.1129032221979622e+20
2    1.1264035744961195e+20
Name: user_id, dtype: object


In [215]:
# Se convierte la columna 'time' a datetime y luego a solo fecha manteniendo tipo datetime
df_california['time'] = pd.to_datetime(df_california['time'], unit='ms').dt.normalize()
print(df_california['time'].head(3))

0   2021-01-06
1   2021-02-09
2   2020-03-08
Name: time, dtype: datetime64[ns]


In [216]:
# Se normaliza la columna 'resp' para agregar su contenido como dos nuevas columnas a df_new_york
resp_df_cal = pd.json_normalize(df_california['resp'])
df_california[['resp_text', 'resp_time']] = resp_df_cal[['text', 'time']]

In [217]:
# Se convierte la columna 'time' a datetime y luego a solo fecha manteniendo tipo datetime
df_california['resp_time'] = pd.to_datetime(df_california['resp_time'], unit='ms').dt.normalize()
print(df_california['resp_time'].head(3))

0   NaT
1   NaT
2   NaT
Name: resp_time, dtype: datetime64[ns]


In [218]:
# Se elimina la columna 'resp' original
df_california = df_california.drop(columns=['resp'])

In [219]:
# Se renombra la columna gmap_id como 'id_negocio', para mayor comprensión en el análisis
df_california = df_california.rename(columns={'gmap_id': 'id_negocio'})

In [220]:
# Se reordena la columna 'id_negocio'
columnas = list(df_california.columns)
columnas.insert(0, columnas.pop(columnas.index('id_negocio')))
df_california = df_california[columnas]

In [221]:
# Se añade la columna 'review_id' autoincremental desde 2700001 para continuar desde el df_new_york
df_california.insert(0, 'review_id', range(2700001, 2700001 + len(df_california)))

In [222]:
# Se añade la columna 'Estado'
df_california.insert(2, 'estado', 'California')

In [223]:
# Se verifica como queda el dataframe
print(df_california.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2700000 entries, 0 to 2699999
Data columns (total 10 columns):
 #   Column      Dtype         
---  ------      -----         
 0   review_id   int64         
 1   id_negocio  object        
 2   estado      object        
 3   user_id     object        
 4   name        object        
 5   time        datetime64[ns]
 6   rating      int64         
 7   text        object        
 8   resp_text   object        
 9   resp_time   datetime64[ns]
dtypes: datetime64[ns](2), int64(2), object(6)
memory usage: 206.0+ MB
None


# Pennsylvania

Se convierten los archivos Json a formato parquet

In [224]:
# Carpeta de entrada que contiene los archivos JSON
input_folder = r'C:\Users\debor\OneDrive\Documentos\Henry\PROYECTO FINAL local\Datasets\datasets_google\estados_json\NORESTE\review-Pennsylvania'
# Carpeta de salida para guardar los archivos Parquet
output_folder = r'C:\Users\debor\OneDrive\Documentos\Henry\PROYECTO FINAL local\Datasets\datasets_google\estados_parquet\NORESTE_parquet\Rev-Pennsylvania.parquet'

# Crear la carpeta de salida si no existe
os.makedirs(output_folder, exist_ok=True)

# Iterar sobre todos los archivos en la carpeta de entrada
for filename in os.listdir(input_folder):
    if filename.endswith('.json'):
        # Ruta completa del archivo JSON
        json_file_path = os.path.join(input_folder, filename)
        
        # Leer el archivo JSON
        try:
            df = pd.read_json(json_file_path, lines=True)
            
            # Crear nombre de archivo Parquet (sin la extensión .json)
            parquet_file_path = os.path.join(output_folder, filename.replace('.json', '.parquet'))
            
            # Guardar el DataFrame como archivo Parquet
            df.to_parquet(parquet_file_path, index=False, compression='snappy', engine='pyarrow')
            
            print(f"Convertido: {filename} a {parquet_file_path}")
        except Exception as e:
            print(f"Error al convertir {filename}: {e}")

print("Conversión completada.")

Convertido: 1.json a C:\Users\debor\OneDrive\Documentos\Henry\PROYECTO FINAL local\Datasets\datasets_google\estados_parquet\NORESTE_parquet\Rev-Pennsylvania.parquet\1.parquet
Convertido: 10.json a C:\Users\debor\OneDrive\Documentos\Henry\PROYECTO FINAL local\Datasets\datasets_google\estados_parquet\NORESTE_parquet\Rev-Pennsylvania.parquet\10.parquet
Convertido: 11.json a C:\Users\debor\OneDrive\Documentos\Henry\PROYECTO FINAL local\Datasets\datasets_google\estados_parquet\NORESTE_parquet\Rev-Pennsylvania.parquet\11.parquet
Convertido: 12.json a C:\Users\debor\OneDrive\Documentos\Henry\PROYECTO FINAL local\Datasets\datasets_google\estados_parquet\NORESTE_parquet\Rev-Pennsylvania.parquet\12.parquet
Convertido: 13.json a C:\Users\debor\OneDrive\Documentos\Henry\PROYECTO FINAL local\Datasets\datasets_google\estados_parquet\NORESTE_parquet\Rev-Pennsylvania.parquet\13.parquet
Convertido: 14.json a C:\Users\debor\OneDrive\Documentos\Henry\PROYECTO FINAL local\Datasets\datasets_google\estados_

In [225]:
# Leer el archivo Parquet
Rev_Pennsylvania_1 = pd.read_parquet(r'C:\Users\debor\OneDrive\Documentos\Henry\PROYECTO FINAL local\Datasets\datasets_google\estados_parquet\NORESTE_parquet\Rev-Pennsylvania.parquet')

print(Rev_Pennsylvania_1.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400000 entries, 0 to 2399999
Data columns (total 8 columns):
 #   Column   Dtype  
---  ------   -----  
 0   user_id  float64
 1   name     object 
 2   time     int64  
 3   rating   int64  
 4   text     object 
 5   pics     object 
 6   resp     object 
 7   gmap_id  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 146.5+ MB
None


Se concatenan los archivos parquet para formar un solo dataframe

In [226]:
# Carpeta que contiene los archivos Parquet
input_folder_parquet = r'C:\Users\debor\OneDrive\Documentos\Henry\PROYECTO FINAL local\Datasets\datasets_google\estados_parquet\NORESTE_parquet\Rev-Pennsylvania.parquet'

# Lista para almacenar DataFrames
df_list_Pennsylvania = []

# Iterar sobre todos los archivos Parquet en la carpeta de entrada
for filename in os.listdir(input_folder_parquet):
    if filename.endswith('.parquet'):
        # Ruta completa del archivo Parquet
        parquet_file_path_psv = os.path.join(input_folder_parquet, filename)
        
        # Leer el archivo Parquet y añadir el DataFrame a la lista
        df = pd.read_parquet(parquet_file_path_psv)
        df_list_Pennsylvania.append(df)

# Concatenar todos los DataFrames en uno solo
df_pennsylvania = pd.concat(df_list_Pennsylvania, ignore_index=True)

# Verificar el DataFrame combinado
print(df_pennsylvania.shape)
print

(2400000, 8)


<function print(*args, sep=' ', end='\n', file=None, flush=False)>

Se elimina columna irrelevante para el análisis

In [227]:
df_pennsylvania = df_pennsylvania.drop(columns=['pics'])

Se convierten las columnas necesarias

In [228]:
# Se convierte la columna 'user_id' a tipo object
df_pennsylvania['user_id'] = df_pennsylvania['user_id'].astype('str')
print(df_pennsylvania['user_id'].head(3))

0    1.0494474255907976e+20
1    1.1760970283298033e+20
2    1.1056324201842663e+20
Name: user_id, dtype: object


In [229]:
# Se convierte la columna 'time' a datetime y luego a solo fecha manteniendo tipo datetime
df_pennsylvania['time'] = pd.to_datetime(df_pennsylvania['time'], unit='ms').dt.normalize()
print(df_pennsylvania['time'].head(3))

0   2018-02-04
1   2016-10-12
2   2012-06-04
Name: time, dtype: datetime64[ns]


In [230]:
# Se normaliza la columna 'resp' para agregar su contenido como dos nuevas columnas a df_new_york
resp_df_pen = pd.json_normalize(df_pennsylvania['resp'])
df_pennsylvania[['resp_text', 'resp_time']] = resp_df_pen[['text', 'time']]

In [231]:
# Se convierte la columna 'resp_time' a datetime y luego a solo fecha manteniendo tipo datetime
df_pennsylvania['resp_time'] = pd.to_datetime(df_pennsylvania['resp_time'], unit='ms').dt.normalize()
print(df_pennsylvania['resp_time'].head(3))

0   NaT
1   NaT
2   NaT
Name: resp_time, dtype: datetime64[ns]


In [232]:
# Se elimina la columna 'resp' original
df_pennsylvania = df_pennsylvania.drop(columns=['resp'])

In [233]:
# Se renombra la columna gmap_id como 'id_negocio', para mayor comprensión en el análisis
df_pennsylvania = df_pennsylvania.rename(columns={'gmap_id': 'id_negocio'})

In [234]:
# Se reordena la columna 'id_negocio'
columnas = list(df_pennsylvania.columns)
columnas.insert(0, columnas.pop(columnas.index('id_negocio')))
df_pennsylvania = df_pennsylvania[columnas]

In [235]:
# Se añade la columna 'review_id' autoincremental desde 4500001 para continuar desde el df_california
df_pennsylvania.insert(0, 'review_id', range(4500001, 4500001 + len(df_pennsylvania)))

In [236]:
# Se añade la columna 'Estado'
df_pennsylvania.insert(2, 'estado', 'Pennsylvania')

In [237]:
# Se verifica como queda el dataframe
print(df_pennsylvania.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400000 entries, 0 to 2399999
Data columns (total 10 columns):
 #   Column      Dtype         
---  ------      -----         
 0   review_id   int64         
 1   id_negocio  object        
 2   estado      object        
 3   user_id     object        
 4   name        object        
 5   time        datetime64[ns]
 6   rating      int64         
 7   text        object        
 8   resp_text   object        
 9   resp_time   datetime64[ns]
dtypes: datetime64[ns](2), int64(2), object(6)
memory usage: 183.1+ MB
None


# Florida

Se convierten los archivos Json a formato parquet

In [238]:
# Carpeta de entrada que contiene los archivos JSON
input_folder = r'C:\Users\debor\OneDrive\Documentos\Henry\PROYECTO FINAL local\Datasets\datasets_google\estados_json\SUR\review-Florida'
# Carpeta de salida para guardar los archivos Parquet
output_folder = r'C:\Users\debor\OneDrive\Documentos\Henry\PROYECTO FINAL local\Datasets\datasets_google\estados_parquet\SUR_parquet\Rev-Florida.parquet'

# Crear la carpeta de salida si no existe
os.makedirs(output_folder, exist_ok=True)

# Iterar sobre todos los archivos en la carpeta de entrada
for filename in os.listdir(input_folder):
    if filename.endswith('.json'):
        # Ruta completa del archivo JSON
        json_file_path = os.path.join(input_folder, filename)
        
        # Leer el archivo JSON
        try:
            df = pd.read_json(json_file_path, lines=True)
            
            # Crear nombre de archivo Parquet (sin la extensión .json)
            parquet_file_path = os.path.join(output_folder, filename.replace('.json', '.parquet'))
            
            # Guardar el DataFrame como archivo Parquet
            df.to_parquet(parquet_file_path, index=False, compression='snappy', engine='pyarrow')
            
            print(f"Convertido: {filename} a {parquet_file_path}")
        except Exception as e:
            print(f"Error al convertir {filename}: {e}")

print("Conversión completada.")

Convertido: 1.json a C:\Users\debor\OneDrive\Documentos\Henry\PROYECTO FINAL local\Datasets\datasets_google\estados_parquet\SUR_parquet\Rev-Florida.parquet\1.parquet
Convertido: 10.json a C:\Users\debor\OneDrive\Documentos\Henry\PROYECTO FINAL local\Datasets\datasets_google\estados_parquet\SUR_parquet\Rev-Florida.parquet\10.parquet
Convertido: 11.json a C:\Users\debor\OneDrive\Documentos\Henry\PROYECTO FINAL local\Datasets\datasets_google\estados_parquet\SUR_parquet\Rev-Florida.parquet\11.parquet
Convertido: 12.json a C:\Users\debor\OneDrive\Documentos\Henry\PROYECTO FINAL local\Datasets\datasets_google\estados_parquet\SUR_parquet\Rev-Florida.parquet\12.parquet
Convertido: 13.json a C:\Users\debor\OneDrive\Documentos\Henry\PROYECTO FINAL local\Datasets\datasets_google\estados_parquet\SUR_parquet\Rev-Florida.parquet\13.parquet
Convertido: 14.json a C:\Users\debor\OneDrive\Documentos\Henry\PROYECTO FINAL local\Datasets\datasets_google\estados_parquet\SUR_parquet\Rev-Florida.parquet\14.pa

In [239]:
# Leer el archivo Parquet
Rev_Florida_1 = pd.read_parquet(r'C:\Users\debor\OneDrive\Documentos\Henry\PROYECTO FINAL local\Datasets\datasets_google\estados_parquet\SUR_parquet\Rev-Florida.parquet')

print(Rev_Florida_1.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2850000 entries, 0 to 2849999
Data columns (total 8 columns):
 #   Column   Dtype  
---  ------   -----  
 0   user_id  float64
 1   name     object 
 2   time     int64  
 3   rating   int64  
 4   text     object 
 5   pics     object 
 6   resp     object 
 7   gmap_id  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 174.0+ MB
None


Se concatenan los archivos parquet para formar un solo dataframe

In [240]:
# Carpeta que contiene los archivos Parquet
input_folder_parquet = r'C:\Users\debor\OneDrive\Documentos\Henry\PROYECTO FINAL local\Datasets\datasets_google\estados_parquet\SUR_parquet\Rev-Florida.parquet'

# Lista para almacenar DataFrames
df_list_Florida = []

# Iterar sobre todos los archivos Parquet en la carpeta de entrada
for filename in os.listdir(input_folder_parquet):
    if filename.endswith('.parquet'):
        # Ruta completa del archivo Parquet
        parquet_file_path_psv = os.path.join(input_folder_parquet, filename)
        
        # Leer el archivo Parquet y añadir el DataFrame a la lista
        df = pd.read_parquet(parquet_file_path_psv)
        df_list_Florida.append(df)

# Concatenar todos los DataFrames en uno solo
df_florida = pd.concat(df_list_Florida, ignore_index=True)

# Verificar el DataFrame combinado
print(df_florida.shape)
print

(2850000, 8)


<function print(*args, sep=' ', end='\n', file=None, flush=False)>

Se elimina columna irrelevante para el análisis

In [241]:
df_florida = df_florida.drop(columns=['pics'])

Se convierten las columnas necesarias

In [242]:
# Se convierte la columna 'user_id' a tipo object
df_florida['user_id'] = df_florida['user_id'].astype('str')
print(df_florida['user_id'].head(3))

0    1.0147185615514873e+20
1    1.1547723478903833e+20
2    1.0180501024489284e+20
Name: user_id, dtype: object


In [243]:
# Se convierte la columna 'time' a datetime y luego a solo fecha manteniendo tipo datetime
df_florida['time'] = pd.to_datetime(df_florida['time'], unit='ms').dt.normalize()
print(df_florida['time'].head(3))

0   2021-08-03
1   2020-07-18
2   2018-04-05
Name: time, dtype: datetime64[ns]


In [244]:
# Se normaliza la columna 'resp' para agregar su contenido como dos nuevas columnas a df_new_york
resp_df_fl = pd.json_normalize(df_florida['resp'])
df_florida[['resp_text', 'resp_time']] = resp_df_fl[['text', 'time']]

In [245]:
# Se convierte la columna 'resp_time' a datetime y luego a solo fecha manteniendo tipo datetime
df_florida['resp_time'] = pd.to_datetime(df_florida['resp_time'], unit='ms').dt.normalize()
print(df_florida['resp_time'].head(3))

0   2021-07-23
1   2020-02-23
2          NaT
Name: resp_time, dtype: datetime64[ns]


In [246]:
# Se elimina la columna 'resp' original
df_florida = df_florida.drop(columns=['resp'])

In [247]:
# Se renombra la columna gmap_id como 'id_negocio', para mayor comprensión en el análisis
df_florida = df_florida.rename(columns={'gmap_id': 'id_negocio'})

In [248]:
# Se reordena la columna 'id_negocio'
columnas = list(df_florida.columns)
columnas.insert(0, columnas.pop(columnas.index('id_negocio')))
df_florida = df_florida[columnas]

In [249]:
# Se añade la columna 'review_id' autoincremental desde 6900001 para continuar desde el df_pennsylvania
df_florida.insert(0, 'review_id', range(6900001, 6900001 + len(df_florida)))

In [250]:
# Se añade la columna 'estado'
df_florida.insert(2, 'estado', 'Pennsylvania')

In [251]:
# Se verifica como queda el dataframe
print(df_florida.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2850000 entries, 0 to 2849999
Data columns (total 10 columns):
 #   Column      Dtype         
---  ------      -----         
 0   review_id   int64         
 1   id_negocio  object        
 2   estado      object        
 3   user_id     object        
 4   name        object        
 5   time        datetime64[ns]
 6   rating      int64         
 7   text        object        
 8   resp_text   object        
 9   resp_time   datetime64[ns]
dtypes: datetime64[ns](2), int64(2), object(6)
memory usage: 217.4+ MB
None


In [252]:
df_florida.tail()

,review_id,id_negocio,estado,user_id,name,time,rating,text,resp_text,resp_time
2849995,9749996,0x8890966585e36d3f:0x131d47c2c60a8d31,Pennsylvania,1.0028771480080918e+20,James Rudolph,2021-04-01,5,None,Thank you!,2021-04-04
2849996,9749997,0x8890966585e36d3f:0x131d47c2c60a8d31,Pennsylvania,1.1409893309742336e+20,Vincent Alexander,2020-02-16,3,None,NaN,NaT
2849997,9749998,0x8890966585e36d3f:0x131d47c2c60a8d31,Pennsylvania,1.1234480481217744e+20,Brett Owen,2019-02-27,5,None,NaN,NaT
2849998,9749999,0x8890966585e36d3f:0x131d47c2c60a8d31,Pennsylvania,1.0544647183143027e+20,ashly kindle,2018-03-15,5,None,"Thank you, Ashly",2018-01-25
2849999,9750000,0x8890966585e36d3f:0x131d47c2c60a8d31,Pennsylvania,1.0874328165743621e+20,Jomarra Auge,2018-09-09,5,None,"Thank you, Mrs. Jomarra:)",2018-09-09


# Texas

Se convierten los archivos Json a formato parquet

In [253]:
# Carpeta de entrada que contiene los archivos JSON
input_folder = r'C:\Users\debor\OneDrive\Documentos\Henry\PROYECTO FINAL local\Datasets\datasets_google\estados_json\SUR\review-Texas'
# Carpeta de salida para guardar los archivos Parquet
output_folder = r'C:\Users\debor\OneDrive\Documentos\Henry\PROYECTO FINAL local\Datasets\datasets_google\estados_parquet\SUR_parquet\Rev-Texas.parquet'

# Crear la carpeta de salida si no existe
os.makedirs(output_folder, exist_ok=True)

# Iterar sobre todos los archivos en la carpeta de entrada
for filename in os.listdir(input_folder):
    if filename.endswith('.json'):
        # Ruta completa del archivo JSON
        json_file_path = os.path.join(input_folder, filename)
        
        # Leer el archivo JSON
        try:
            df = pd.read_json(json_file_path, lines=True)
            
            # Crear nombre de archivo Parquet (sin la extensión .json)
            parquet_file_path = os.path.join(output_folder, filename.replace('.json', '.parquet'))
            
            # Guardar el DataFrame como archivo Parquet
            df.to_parquet(parquet_file_path, index=False, compression='snappy', engine='pyarrow')
            
            print(f"Convertido: {filename} a {parquet_file_path}")
        except Exception as e:
            print(f"Error al convertir {filename}: {e}")

print("Conversión completada.")

Convertido: 1.json a C:\Users\debor\OneDrive\Documentos\Henry\PROYECTO FINAL local\Datasets\datasets_google\estados_parquet\SUR_parquet\Rev-Texas.parquet\1.parquet
Convertido: 10.json a C:\Users\debor\OneDrive\Documentos\Henry\PROYECTO FINAL local\Datasets\datasets_google\estados_parquet\SUR_parquet\Rev-Texas.parquet\10.parquet
Convertido: 11.json a C:\Users\debor\OneDrive\Documentos\Henry\PROYECTO FINAL local\Datasets\datasets_google\estados_parquet\SUR_parquet\Rev-Texas.parquet\11.parquet
Convertido: 12.json a C:\Users\debor\OneDrive\Documentos\Henry\PROYECTO FINAL local\Datasets\datasets_google\estados_parquet\SUR_parquet\Rev-Texas.parquet\12.parquet
Convertido: 13.json a C:\Users\debor\OneDrive\Documentos\Henry\PROYECTO FINAL local\Datasets\datasets_google\estados_parquet\SUR_parquet\Rev-Texas.parquet\13.parquet
Convertido: 14.json a C:\Users\debor\OneDrive\Documentos\Henry\PROYECTO FINAL local\Datasets\datasets_google\estados_parquet\SUR_parquet\Rev-Texas.parquet\14.parquet
Conver

In [254]:
# Leer el archivo Parquet
Rev_Texas_1 = pd.read_parquet(r'C:\Users\debor\OneDrive\Documentos\Henry\PROYECTO FINAL local\Datasets\datasets_google\estados_parquet\SUR_parquet\Rev-Texas.parquet')

print(Rev_Texas_1.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2296824 entries, 0 to 2296823
Data columns (total 8 columns):
 #   Column   Dtype  
---  ------   -----  
 0   user_id  float64
 1   name     object 
 2   time     int64  
 3   rating   int64  
 4   text     object 
 5   pics     object 
 6   resp     object 
 7   gmap_id  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 140.2+ MB
None


Se concatenan los archivos parquet para formar un solo dataframe

In [255]:
# Carpeta que contiene los archivos Parquet
input_folder_parquet = r'C:\Users\debor\OneDrive\Documentos\Henry\PROYECTO FINAL local\Datasets\datasets_google\estados_parquet\NORESTE_parquet\Rev-Pennsylvania.parquet'

# Lista para almacenar DataFrames
df_list_Texas = []

# Iterar sobre todos los archivos Parquet en la carpeta de entrada
for filename in os.listdir(input_folder_parquet):
    if filename.endswith('.parquet'):
        # Ruta completa del archivo Parquet
        parquet_file_path_psv = os.path.join(input_folder_parquet, filename)
        
        # Leer el archivo Parquet y añadir el DataFrame a la lista
        df = pd.read_parquet(parquet_file_path_psv)
        df_list_Texas.append(df)

# Concatenar todos los DataFrames en uno solo
df_texas = pd.concat(df_list_Texas, ignore_index=True)

# Verificar el DataFrame combinado
print(df_texas.shape)
print

(2400000, 8)


<function print(*args, sep=' ', end='\n', file=None, flush=False)>

Se elimina columna irrelevante para el análisis

In [256]:
df_texas = df_texas.drop(columns=['pics'])

Se convierten las columnas necesarias

In [257]:
# Se convierte la columna 'user_id' a tipo object
df_texas['user_id'] = df_texas['user_id'].astype('str')
print(df_texas['user_id'].head(3))

0    1.0494474255907976e+20
1    1.1760970283298033e+20
2    1.1056324201842663e+20
Name: user_id, dtype: object


In [258]:
# Se convierte la columna 'time' a datetime y luego a solo fecha manteniendo tipo datetime
df_texas['time'] = pd.to_datetime(df_texas['time'], unit='ms').dt.normalize()
print(df_texas['time'].head(3))

0   2018-02-04
1   2016-10-12
2   2012-06-04
Name: time, dtype: datetime64[ns]


In [259]:
# Se normaliza la columna 'resp' para agregar su contenido como dos nuevas columnas a df_new_york
resp_df_tx = pd.json_normalize(df_texas['resp'])
df_texas[['resp_text', 'resp_time']] = resp_df_tx[['text', 'time']]

In [260]:
# Se convierte la columna 'resp_time' a datetime y luego a solo fecha manteniendo tipo datetime
df_texas['resp_time'] = pd.to_datetime(df_texas['resp_time'], unit='ms').dt.normalize()
print(df_texas['resp_time'].head(3))

0   NaT
1   NaT
2   NaT
Name: resp_time, dtype: datetime64[ns]


In [261]:
# Se elimina la columna 'resp' original
df_texas = df_texas.drop(columns=['resp'])

In [262]:
# Se renombra la columna gmap_id como 'id_negocio', para mayor comprensión en el análisis
df_texas = df_texas.rename(columns={'gmap_id': 'id_negocio'})

In [263]:
# Se reordena la columna 'id_negocio'
columnas = list(df_texas.columns)
columnas.insert(0, columnas.pop(columnas.index('id_negocio')))
df_texas = df_texas[columnas]

In [264]:
# Se añade la columna 'review_id' autoincremental desde 9750001 para continuar desde el df_florida
df_texas.insert(0, 'review_id', range(9750001, 9750001 + len(df_texas)))

In [265]:
# Se añade la columna 'estado'
df_texas.insert(2, 'estado', 'Pennsylvania')

In [266]:
# Se verifica como queda el dataframe
print(df_texas.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400000 entries, 0 to 2399999
Data columns (total 10 columns):
 #   Column      Dtype         
---  ------      -----         
 0   review_id   int64         
 1   id_negocio  object        
 2   estado      object        
 3   user_id     object        
 4   name        object        
 5   time        datetime64[ns]
 6   rating      int64         
 7   text        object        
 8   resp_text   object        
 9   resp_time   datetime64[ns]
dtypes: datetime64[ns](2), int64(2), object(6)
memory usage: 183.1+ MB
None


In [267]:
# Se concatenan los  DataFrames anteriores para formar uno solo
g_estados_seleccionados = pd.concat([df_new_york, df_california, df_pennsylvania, df_florida, df_texas])

In [268]:
g_estados_seleccionados_2 = g_estados_seleccionados.to_parquet(r'C:\Users\debor\OneDrive\Documentos\Henry\PROYECTO FINAL local\Datasets\datasets_google\g_estados_seleccionados_2.parquet', engine='auto', compression='snappy')

In [269]:
print(g_estados_seleccionados.info())

<class 'pandas.core.frame.DataFrame'>
Index: 13050000 entries, 0 to 2399999
Data columns (total 10 columns):
 #   Column      Dtype         
---  ------      -----         
 0   review_id   int64         
 1   id_negocio  object        
 2   estado      object        
 3   user_id     object        
 4   name        object        
 5   time        datetime64[ns]
 6   rating      int64         
 7   text        object        
 8   resp_text   object        
 9   resp_time   datetime64[ns]
dtypes: datetime64[ns](2), int64(2), object(6)
memory usage: 1.1+ GB
None


In [270]:
nulos_por_columna = g_estados_seleccionados.isnull().sum()
print(nulos_por_columna)

review_id            0
id_negocio           0
estado               0
user_id              0
name                 0
time                 0
rating               0
text           5496839
resp_text     11485213
resp_time     11485213
dtype: int64


In [271]:
g_sitios = pd.read_parquet(r'C:\Users\debor\OneDrive\Documentos\Henry\PROYECTO FINAL local\Datasets\datasets_google\g_sitios_v3.parquet')

In [272]:
# Obtenemos los ID unicos
g_sitios_ids = g_sitios['id_negocio'].unique()

# Filtrar las tablas por los IDs que coincidan con los de g_sitios
g_estados_seleccionados = g_estados_seleccionados[g_estados_seleccionados['id_negocio'].isin(g_sitios_ids)]

In [273]:
# Resetear los índices del DataFrame filtrado
g_estados_seleccionados = g_estados_seleccionados.reset_index(drop=True)

# Crear una nueva columna 'review_id' con valores consecutivos solo para el DataFrame filtrado
g_estados_seleccionados['review_id'] = range(1, len(g_estados_seleccionados) + 1)

In [274]:
g_estados_seleccionados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 258598 entries, 0 to 258597
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   review_id   258598 non-null  int64         
 1   id_negocio  258598 non-null  object        
 2   estado      258598 non-null  object        
 3   user_id     258598 non-null  object        
 4   name        258598 non-null  object        
 5   time        258598 non-null  datetime64[ns]
 6   rating      258598 non-null  int64         
 7   text        150731 non-null  object        
 8   resp_text   38007 non-null   object        
 9   resp_time   38007 non-null   datetime64[ns]
dtypes: datetime64[ns](2), int64(2), object(6)
memory usage: 19.7+ MB


In [275]:
g_estados_seleccionados.to_parquet(r'C:\Users\debor\OneDrive\Documentos\Henry\PROYECTO FINAL local\Datasets\datasets_google\g_estados_seleccionados.parquet', engine='auto', compression='snappy')